In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [1]:
import torch.nn.functional as F
from fastai import datasets
import gzip, pickle, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

In [2]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train, y_train, x_valid, y_valid = get_data()